In [25]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet import Prophet
import itertools
from sktime.performance_metrics.forecasting import (
    mean_absolute_scaled_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
import time
import holidays
import datetime


# define functions

In [26]:
# preprocessing_endo
def preprocessing_endo(ts, horizon, split=True):
    fb_df = ts[["sales"]]
    fb_df["sales"] = ts["sales"] / 1e6
    fb_df = fb_df.reset_index().rename({"date": "ds", "sales": "y"}, axis=1)

    if split:
        fb_train = fb_df.iloc[:-horizon]
        fb_test = fb_df.iloc[-horizon:]
        return {"y_train": fb_train, "y_test": fb_test}
    else:
        return fb_df


In [27]:
# preprocessing_exog
def preprocessing_exog(df_exog):
    df_promo = pd.DataFrame({
        "holiday": "promo_day",
        "ds": df_exog[df_exog["promo_day"] == 1]["promo_day"].index})
    df_off = pd.DataFrame({
        "holiday": "off_day",
        "ds": df_exog[df_exog["off_day"] == 1]["off_day"].index})
    return pd.concat([df_promo, df_off])


In [28]:
# grid seach Prophet
def grid_search_prophet(prophet_data_endo, prophet_data_exog, param_grid):
    # create df to store result
    mape_PROPHET = pd.DataFrame(columns=[
        "changepoint_prior_scale", 
        "seasonality_prior_scale",
        "holidays_prior_scale",
        "mape",
        "mae",
        "rmse",
        ])
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

    # grid search
    y_train = prophet_data_endo['y_train']
    y_test = prophet_data_endo['y_test']
    for params in all_params:
        m = Prophet(holidays=prophet_data_exog,
                    **params).fit(y_train)  # Fit model with given params
        y_hat = m.predict(y_test)['yhat']
        mape_PROPHET = mape_PROPHET.append(
            {"changepoint_prior_scale": params["changepoint_prior_scale"], 
             "seasonality_prior_scale": params["seasonality_prior_scale"], 
             "holidays_prior_scale": params["holidays_prior_scale"], 
             "mape": round(mean_absolute_percentage_error(y_test['y'], y_hat), 3),
             "mae": round(mean_absolute_error(y_test['y'], y_hat), 3),
             "rmse": round(mean_squared_error(y_test['y'], y_hat, square_root=True), 3),
             }, ignore_index=True)
    # mape_PROPHET.sort_values('mape', inplace=True)
    return mape_PROPHET


In [32]:
# CV 
def cross_validation_result(data, exog, params, model_name, horizon, rolls=4):

    mae_CVs = []
    rmse_CVs = []
    mape_CVs = []
    mase_CVs = []
    for i in range(rolls):
        print(f"fold {i}---------------")
        
        # split data
        y_train = data.iloc[: -(rolls - i) * horizon]
        y_test = data.iloc[
            np.r_[-(rolls - i) * horizon : -(rolls - i - 1) * horizon]]

        # fit model
        model = Prophet(**params, holidays=exog)
        model.fit(y_train)
        
        # make forecast
        y_hat = model.predict(y_test)['yhat']

        # score
        mae_CVs.append(round(mean_absolute_error(y_test['y'], y_hat), 3))
        rmse_CVs.append(round(mean_squared_error(y_test['y'], y_hat, square_root=True), 3))
        mape_CVs.append(round(mean_absolute_percentage_error(y_test['y'], y_hat), 3))
        mase_CVs.append(round(mean_absolute_scaled_error(y_test['y'], y_hat, y_train=y_train['y']), 3))
    
    return {'store':model_name,
           'mae_PROPHET':np.mean(mae_CVs),
           'rmse_PROPHET':np.mean(rmse_CVs),
           'mape_PROPHET':np.mean(mape_CVs),
           'mase_PROPHET':np.mean(mase_CVs),
           'fc_PROPHET':y_hat,
           }


# read data

In [6]:
df_store = pd.read_pickle("data/df_daily.pkl")
df_company = df_store.groupby("date").sum()[["sales"]]
df_exog = pd.read_pickle("data/df_exog.pkl")


# horizon = 7

## tune on company data

In [5]:
horizon = 7
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [20]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.001, 0.01, 0.05, 0.1, 0.5], 
    "seasonality_prior_scale": [0.01, 0.1, 1, 10],
    "holidays_prior_scale": [0.01, 0.1, 1, 10],
    }

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2683.31   3.11993e-05       5191.88      0.3629           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     106       2685.34   0.000236981       5576.32   4.755e-08       0.001      172  LS failed, Hessian reset 
     190       2686.38   9.87338e-09       4874.52      0.6034      0.6034      271   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      92       2732.77   0.000153759       4865.87    3.33e-08       0.001      154  LS failed, Hessian reset 
      99       2733.22   1.53875e-05       4813.37      0.1784           1      163   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     133       2733.45   1.58682e-06       4471.56   3.346e-10       0.001      246  LS failed, Hessian reset 
     152       2733.46   6.14168e-09       3959.31      0.2564      0.2564      268   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2819.68    0.00295478       101.845      0.3069      0.3069      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2821.77     0.0019403       25.8795           1           1      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2822.38     0.0004924       22.4665      0.9938      0.9938      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2822.83   8.30955e-05       6.22544      0.1817           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2823.03   0.000986215       15.5377      0.8148      0.8148      590   
    Iter      log prob        ||dx||      ||grad||       alpha  

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2819.55    0.00325933       43.7645           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2821.38    0.00206414       58.6002     0.07598           1      232   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2822.27   0.000787447       35.9459      0.8456      0.8456      346   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2822.75   0.000486484       27.0489           1           1      467   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2823.09    0.00140344       55.9312      0.5644      0.5644      585   
    Iter      log prob        ||dx||      ||grad||       alpha  

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [46]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.05], 
    "seasonality_prior_scale": [10],
    "holidays_prior_scale": [10],
    }

grid_search_result1 = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2803.75   0.000219649       68.7782      0.7949      0.7949      140   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     126       2804.01     0.0010467       151.622   1.005e-05       0.001      219  LS failed, Hessian reset 
     199       2804.15   1.04472e-05       69.6038           1           1      313   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2804.17   1.57902e-05       78.2213      0.4984      0.4984      433   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2804.21   1.12076e-06       60.7159      0.2379           1      563   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     415       2804.21   6.26992e-06    

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [42]:
grid_search_result.sort_values(['mape', 'rmse'], inplace=True)
# grid_search_result.head(10).to_csv("results/f8/PROPHET_param_7.csv")
grid_search_result#.head(10)


,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,mape,mae,rmse
44,0.1,10.0,0.01,0.1,113.516,177.69
36,0.1,0.1,0.01,0.1,113.312,177.961
40,0.1,1.0,0.01,0.1,113.006,178.123
32,0.1,0.01,0.01,0.101,114.927,180.84
8,0.001,1.0,0.01,0.11,124.677,180.575
...,...,...,...,...,...,...
27,0.01,1.0,10.0,0.174,162.259,195.028
18,0.01,0.01,1.0,0.175,162.689,198.234
26,0.01,1.0,1.0,0.176,163.5,195.8
19,0.01,0.01,10.0,0.176,162.82,198.553


In [47]:
# grid_search_result.query('changepoint_prior_scale==10')
#['seasonality_prior_scale', 'holidays_prior_scale', 'changepoint_prior_scale']
grid_search_result1

,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,mape,mae,rmse
0,0.05,10.0,10.0,0.148,148.398,188.177


In [38]:
# grid_search_result.to_pickle("results/f8/PROPHET_param_7.pkl")
# grid_search_result.to_csv("results/f8/PROPHET_param_7.csv")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_7.pkl")
tuned_mape = grid_search_result.iloc[0, 3]
tuned_prophet_params = grid_search_result.iloc[0,:3].to_dict()

tuned_mape, tuned_prophet_params


(0.1,
 {'changepoint_prior_scale': 0.1,
  'seasonality_prior_scale': 10.0,
  'holidays_prior_scale': 0.01})

## fit on store data

In [14]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)

    # define model
    # model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name, horizon=horizon)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


processing stores 307222...

fold 0---------------


TypeError: type object argument after ** must be a mapping, not float

## result

In [320]:
# all_stores_result_CV.to_pickle("results/f8/PROPHET_result_7.pkl")


# horizon = 14

## tune on company data

In [30]:
horizon = 14
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [9]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "holidays_prior_scale": [0.01, 0.1, 1, 10],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      84       2712.24   0.000114863       728.092   1.393e-07       0.001      147  LS failed, Hessian reset 
      99       2712.27   1.59952e-06        613.84       0.182           1      168   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     128       2712.32   3.74118e-05       728.149   5.108e-08       0.001      250  LS failed, Hessian reset 
     166       2712.34   9.73487e-09       609.001      0.2038           1      309   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2755.05    7.4909e-06       668.198      0.5808      0.5808      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     128       2755.05   2.15233e-06       820.971   3.505e-09       0.001      212  LS failed, Hessian reset 
     143       2755.05   1.53507e-08       458.318      0.2566       0.967      238   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2794.55   0.000297968       42.7491      0.3528      0.3528      116   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     115       2794.75   0.000124632        61.543   1.999e-06       0.001      172  LS failed, Hessian reset 
     149       2794.98   0.000146126       40.2752   4.657e-06       0.001      255  LS failed, Hessian reset 
     189       2794.99   1.31904e-07       35.0129      0.2907      0.7347      313   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2794.56    0.00155807       49.9909           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2795.26   8.49244e-05       36.0022           1           1      246   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     200       2795.26   1.01362e-05       34.4905   2.815e-07       0.001      289  LS failed, Hessian reset 
     220       2795.26   4.66566e-07       38.3147      0.3297      0.3297      312   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [17]:
# grid_search_result.to_pickle("results/f8/PROPHET_param_14.pkl")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_14.pkl")
tuned_prophet_params = grid_search_result.iloc[0,:3].to_dict()
tuned_mape = grid_search_result.iloc[0, 1]


## fit on store data

In [33]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name, horizon=horizon)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.33816
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2638.05   2.56898e-06       753.519           1           1      148   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     130       2638.05   1.91036e-06       561.182   3.113e-09       0.001      224  LS failed, Hessian reset 
     160       2638.05    6.9135e-09        625.86      0.3482      0.3482      264   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

fold 2---------------
Initial log joint probability = -10.38
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        1346.7   8.71737e-08       1010.16      0.4237      0.4237      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     134       1346.71   1.01401e-07       977.684    9.62e-11       0.001      227  LS failed, Hessian reset 
     142       1346.71   5.05509e-09       730.265      0.4159      0.4159      235   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

fold 3---------------
Initial log joint probability = -10.4518
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      81       1374.75   5.80213e-05       894.035   5.993e-08       0.001      149  LS failed, Hessian reset 
      99       1374.83   1.97719e-06       981.201           1           1      175   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     126       1374.83   2.33399e-09       903.646      0.1217      0.1217      210   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(


## result

In [34]:
all_stores_result_CV.describe()


,mae_PROPHET,rmse_PROPHET,mape_PROPHET,mase_PROPHET
count,38.000000,38.000000,38.000000,38.000000
mean,6.641520,8.707921,0.367395,0.901329
std,5.248917,7.751755,0.109675,0.234543
min,2.691750,3.427250,0.228000,0.598000
25%,3.582437,4.493750,0.295625,0.769125
50%,4.863125,6.273000,0.358875,0.843625
75%,7.305500,8.826813,0.400000,0.978438
max,26.843000,38.714250,0.824000,1.902250


In [35]:
all_stores_result_CV.to_pickle("results/f8/PROPHET_result_14.pkl")


# horizon = 21
## tune on company data

In [36]:
horizon = 21
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [37]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "holidays_prior_scale": [0.01, 0.1, 1, 10],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      98          2695   0.000132309         864.7   2.022e-07       0.001      179  LS failed, Hessian reset 
      99       2695.02   7.77652e-05       709.992           1           1      180   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2695.39   7.22287e-08       762.785           1           1      320   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     226       2695.39   3.22806e-06       756.917    4.52e-09       0.001      388  LS failed, Hessian reset 
     244       2695.39   6.83405e-09       633.464      0.3282      0.3282      418   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2737.68   7.94171e-05       742.462       2.367      0.2367      144   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2737.92   7.84252e-05       659.401      0.7447      0.7447      274   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2738.23   4.07742e-06       779.176      0.8611      0.8611      407   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     351       2738.23   6.14374e-09       713.491      0.3301      0.3301      476   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2777.31    0.00377891       112.091           1           1      113   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     162       2777.82   2.23336e-05       38.8818   6.331e-07       0.001      229  LS failed, Hessian reset 
     178       2777.82   4.32457e-07       27.3895       0.453       0.453      250   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2777.38   0.000650045       66.1093      0.5527           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2778.07   8.28763e-05       42.9295      0.4257           1      254   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     288       2778.21   0.000128011       42.2271   3.159e-06       0.001      414  LS failed, Hessian reset 
     299       2778.21   7.62074e-05       26.1917      0.7427      0.7427      426   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2778.27    0.00014366       38.2238           1           1      545   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     440       2778.27   1.54815e-06    

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [41]:
grid_search_result.to_pickle("results/f8/PROPHET_param_21.pkl")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_14.pkl")
tuned_prophet_params = grid_search_result.iloc[0,:3].to_dict()
tuned_mape = grid_search_result.iloc[0, 1]


## fit on store data

In [42]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)
   
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name, horizon=horizon)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.49984
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        2588.5   0.000131152       772.726   1.512e-07       0.001      160  LS failed, Hessian reset 
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     172       2588.63   1.85959e-08       547.886           1           1      250   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

fold 2---------------
Initial log joint probability = -11.8719
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      92       1326.04   3.59577e-05       882.952   3.506e-08       0.001      159  LS failed, Hessian reset 
      99       1326.07    1.3822e-05        1041.9      0.9581      0.9581      167   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     146       1326.08   1.12515e-07       973.505   1.065e-10       0.001      268  LS failed, Hessian reset 
     155       1326.08   8.99907e-09       839.875           1           1      279   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

fold 3---------------
Initial log joint probability = -11.5579
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1361.39   7.01783e-07       999.416      0.2589           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     127       1361.39   6.61541e-09        970.81      0.3307           1      164   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(


## result

In [43]:
all_stores_result_CV.to_pickle("results/f8/PROPHET_result_21.pkl")


# horizon = 28
## tune on company data

In [44]:
horizon = 28
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [45]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "holidays_prior_scale": [0.01, 0.1, 1, 10],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      73       2678.38   0.000284013       860.672   4.042e-07       0.001      134  LS failed, Hessian reset 
      99       2678.46   1.07044e-07       539.206        0.24           1      176   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     139       2678.47   6.02356e-06       448.311   7.487e-09       0.001      272  LS failed, Hessian reset 
     180       2678.48   3.22556e-07       758.897    5.66e-10       0.001      368  LS failed, Hessian reset 
     188       2678.48   3.54782e-09       562.116     0.06166           1      381   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      93       2720.29   0.000267803       732.343   3.527e-07       0.001      160  LS failed, Hessian reset 
      99       2720.48    7.5773e-05       602.975       0.616       0.616      166   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     165       2720.57   1.14805e-06       828.521   1.437e-09       0.001      296  LS failed, Hessian reset 
     183       2720.57   2.12688e-08       579.121           1           1      325   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2760.72    0.00104876       85.5445           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2761.44   0.000186039       24.9097      0.2761      0.2761      249   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     222       2761.54   0.000352848       114.296   4.803e-06       0.001      316  LS failed, Hessian reset 
     251       2761.64   7.72118e-05       40.8291   2.311e-06       0.001      383  LS failed, Hessian reset 
     299       2761.66   2.00304e-06       41.7129      0.5599      0.5599      445   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     309       2761.66    4.1221e-07       33.2164      0.6012      0.6012      457   
Optimization terminate

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2760.65   0.000920255       97.3873           1           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     120        2761.1   0.000146546       70.1836   3.407e-06       0.001      188  LS failed, Hessian reset 
     156       2761.34   5.18265e-05       30.3218   1.666e-06       0.001      264  LS failed, Hessian reset 
     199       2761.35   0.000362034       31.8955      0.6764           1      322   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     234       2761.51   0.000248233       102.144   3.804e-06       0.001      410  LS failed, Hessian reset 
     299       2761.66     6.902e-05        38.964       0.773       0.773      496   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [46]:
grid_search_result.to_pickle("results/f8/PROPHET_param_28.pkl")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_14.pkl")
tuned_prophet_params = grid_search_result.iloc[0,:3].to_dict()
tuned_mape = grid_search_result.iloc[0, 1]


## fit on store data

In [47]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name, horizon=horizon)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.28994
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2514.95   9.17076e-05       785.808           1           1      136   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     134       2515.19   5.74641e-05       706.168   1.058e-07       0.001      258  LS failed, Hessian reset 
     199       2515.33   1.33406e-07       681.354      0.2893           1      343   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     216       2515.33   4.66319e-07       734.552   7.691e-10       0.001      403  LS failed, Hessian reset 
     223       2515.33    1.1367e-08       496.008      0.2766      0.2766      414   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append m

fold 2---------------
Initial log joint probability = -10.8847


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
INFO:prophet:Disabling daily seasonality. Run prophet with 

    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      88       1220.12   5.47394e-05       1068.78   5.128e-08       0.001      155  LS failed, Hessian reset 
      99       1220.16   2.26716e-06       929.216      0.2143           1      169   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     130       1220.16   9.05268e-09       914.374      0.9057      0.9057      210   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
fold 3---------------
Initial log joint probability = -10.38
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        1346.7   8.71737e-08       1010.16      0.4237      0.4237      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     134       1346.71   1.01401e-07       977.684    9.62e-11       0.001      227  LS failed

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(


## result

In [48]:
all_stores_result_CV.to_pickle("results/f8/PROPHET_result_28.pkl")
